In [1]:
%matplotlib inline

# 保存並加載模型

在本單元中，我們將了解如何通過保存、加載和運行模型預測來保持模型狀態。

In [2]:
import torch
import torch.onnx as onnx
import torchvision.models as models

## 保存和加載模型權重

PyTorch 模型將學習到的參數存儲在一個名為“state_dict”的內部狀態字典中。這些可以通過 `torch.save` 方法持久化：



In [3]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'data/model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/lokinfey/.cache/torch/checkpoints/vgg16-397923af.pth
100.0%


To load model weights, you need to create an instance of the same model first, and then load the parameters using the `load_state_dict()` method.



In [4]:
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('data/model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

> **Note:** Be sure to call `model.eval()` method before inferencing to set the dropout and batch normalization layers to evaluation mode. Failing to do this will yield inconsistent inference results.


## 保存和加載帶有形狀的模型

在加載模型權重時，我們需要先實例化模型類，因為該類定義了網絡的結構。我們可能希望將此類的結構與模型一起保存，在這種情況下，我們可以將 `model`（而不是 `model.state_dict()`）傳遞給保存函數：

In [5]:
torch.save(model, 'data/vgg_model.pth')

We can then load the model like this:

In [6]:
model = torch.load('data/vgg_model.pth')

> **Note:** This approach uses Python [pickle](https://docs.python.org/3/library/pickle.html) module when serializing the model, thus it relies on the actual class definition to be available when loading the model.


## 將模型導出到 ONNX

PyTorch 還具有本機 ONNX 導出支持。然而，鑑於 PyTorch 執行圖的動態特性，導出過程必須遍歷執行圖以生成持久化的 ONNX 模型。出於這個原因，應該將適當大小的測試變量傳遞給導出例程（在我們的例子中，我們將創建一個正確大小的虛擬零張量）：

In [7]:
input_image = torch.zeros((1,3,224,224))
onnx.export(model, input_image, 'data/model.onnx')

您可以使用 ONNX 模型做很多事情，包括在不同平台和不同編程語言上運行推理。